In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

In [2]:
df = pd.read_csv('Video_Game_Data.csv', thousands = ',')
df = df.drop(columns='Unnamed: 0')

In [3]:
df.head()

,Game Title,Weekly Sales,All Time Sales,Weeks Out
0,"Pokemon Sun/Moon (3DS)Nintendo, Role-Playing",312949,12347479,8
1,"Grand Theft Auto V (PS4)Take-Two Interactive, ...",162293,13692364,112
2,"Super Mario Maker (3DS)Nintendo, Platform",147107,1979933,6
3,"FIFA 17 (PS4)Electronic Arts, Sports",138161,9719745,15
4,Call of Duty: Infinite Warfare (PS4)Activision...,134299,6329596,10


In [4]:
weekly_sales = list(df.groupby('Game Title').mean()['Weekly Sales'])

In [5]:
all_time_sales = list(df.groupby('Game Title').max()['All Time Sales'])

In [6]:
weeks_out = list(df.groupby('Game Title').max()['Weeks Out'])

In [8]:
game_title = list(df.groupby('Game Title').mean().index)

In [9]:
genre = []
title = []
console = []
publisher = []
title2 = []

for games in game_title:
    genre.append(games.split(',')[1].strip())
    title.append(games.split(',')[0])
    console.append(games.split('(')[-1].split(')')[0])
    publisher.append(games.split('(')[-1].split(')')[-1].split(',')[0])
    title2.append(games.split('(')[0])

In [10]:
df2 = pd.DataFrame(columns= ['Game_Title', 'Genre','Console', 'Publisher', 'Weekly_Sales', 'All_Time_Sales', 'Weeks_Out'])

In [11]:
df2.Game_Title = title2
df2.Console = console
df2.Publisher = publisher
df2.Weekly_Sales = weekly_sales
df2.All_Time_Sales = all_time_sales
df2.Weeks_Out = weeks_out
df2.Genre = genre

In [12]:
df2.head()

,Game_Title,Genre,Console,Publisher,Weekly_Sales,All_Time_Sales,Weeks_Out
0,.hack//G.U. Last Recode,Role-Playing,PS4,Namco Bandai Games,86008.000000,86008,1
1,1-2-Switch,Party,NS,Nintendo,32010.285714,1927365,62
2,A Way Out,Action,PS4,Electronic Arts,46691.750000,186767,4
3,A Way Out,Action,XOne,Electronic Arts,42659.000000,42659,1
4,ABZU,Adventure,PS4,505 Games,24841.000000,24841,1


In [16]:
df2 = df2[df2['Weeks_Out'] >= 5]

In [71]:
df2 = df2.replace({'Console': "Pokemon: Let's Go"}, {'Console': 'NS'})
df2 = df2.replace({'Publisher': "Pokemon: Let's Go"}, {'Publisher': 'Nintendo'})
df2 = df2.replace({'Genre': "Pikachu! (NS)Nintendo"}, {'Genre': 'RolePlaying'})
df2 = df2.replace({'Genre': "Eevee! (NS)Nintendo"}, {'Genre': 'RolePlaying'})

In [72]:
df2.groupby('Genre').sum()

KeyError: 'Genre'

In [19]:
pub_dummies = pd.get_dummies(df2["Publisher"], prefix="pub")
gen_dummies = pd.get_dummies(df2["Genre"], prefix="gen")
con_dummies = pd.get_dummies(df2["Console"], prefix="con")

In [20]:
df2 = df2.drop(["Publisher", "Genre", "Console"], axis=1)
df2 = pd.concat([df2, pub_dummies, gen_dummies, con_dummies], axis=1)
df2.head()

,Game_Title,Weekly_Sales,All_Time_Sales,Weeks_Out,pub_505 Games,pub_Activision,pub_Atlus,pub_Bandai Namco Entertainment,pub_Bethesda Softworks,pub_Capcom,...,gen_Sports,gen_Strategy,con_3DS,con_NS,con_PC,con_PS3,con_PS4,con_PSV,con_WiiU,con_XOne
1,1-2-Switch,32010.285714,1927365,62,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,ARK: Survival Evolved,47038.250000,270351,10,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11,Akiba's Beat,16362.000000,27914,23,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
13,Animal Crossing: New Leaf,26383.000000,9743459,218,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
14,Arms,65067.933333,1506781,31,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [37]:
cleaned_dummies = []
for names in list(df2.columns)[5:]:
    cleaned_dummies.append(names.replace(' ', '_'))
    
dict(zip(list(df2.columns[5:]), cleaned_dummies))

{'pub_Activision': 'pub_Activision',
 'pub_Atlus': 'pub_Atlus',
 'pub_Bandai_Namco_Entertainment': 'pub_Bandai_Namco_Entertainment',
 'pub_Bethesda_Softworks': 'pub_Bethesda_Softworks',
 'pub_Capcom': 'pub_Capcom',
 'pub_Codemasters': 'pub_Codemasters',
 'pub_Deep_Silver': 'pub_Deep_Silver',
 'pub_Electronic_Arts': 'pub_Electronic_Arts',
 'pub_Gearbox_Software': 'pub_Gearbox_Software',
 'pub_Koch_Media': 'pub_Koch_Media',
 'pub_Konami_Digital_Entertainment': 'pub_Konami_Digital_Entertainment',
 'pub_Level_5': 'pub_Level_5',
 'pub_Marvelous_Interactive': 'pub_Marvelous_Interactive',
 'pub_Maximum_Games': 'pub_Maximum_Games',
 'pub_Microsoft_Game_Studios': 'pub_Microsoft_Game_Studios',
 'pub_Microsoft_Studios': 'pub_Microsoft_Studios',
 'pub_Mojang': 'pub_Mojang',
 'pub_Namco_Bandai_Games': 'pub_Namco_Bandai_Games',
 'pub_Natsume': 'pub_Natsume',
 'pub_Nintendo': 'pub_Nintendo',
 'pub_Nippon_Ichi_Software': 'pub_Nippon_Ichi_Software',
 'pub_PQube': 'pub_PQube',
 'pub_Sega': 'pub_Sega',
 

In [50]:
df2 = df2.rename(columns={'pub_Activision': 'pub_Activision',
 'pub_Atlus': 'pub_Atlus',
 'pub_Bandai_Namco_Entertainment': 'pub_Bandai_Namco_Entertainment',
 'pub_Bethesda_Softworks': 'pub_Bethesda_Softworks',
 'pub_Capcom': 'pub_Capcom',
 'pub_Codemasters': 'pub_Codemasters',
 'pub_Deep_Silver': 'pub_Deep_Silver',
 'pub_Electronic_Arts': 'pub_Electronic_Arts',
 'pub_Gearbox_Software': 'pub_Gearbox_Software',
 'pub_Koch_Media': 'pub_Koch_Media',
 'pub_Konami_Digital_Entertainment': 'pub_Konami_Digital_Entertainment',
 'pub_Level_5': 'pub_Level_5',
 'pub_Marvelous_Interactive': 'pub_Marvelous_Interactive',
 'pub_Maximum_Games': 'pub_Maximum_Games',
 'pub_Microsoft_Game_Studios': 'pub_Microsoft_Game_Studios',
 'pub_Microsoft_Studios': 'pub_Microsoft_Studios',
 'pub_Mojang': 'pub_Mojang',
 'pub_Namco_Bandai_Games': 'pub_Namco_Bandai_Games',
 'pub_Natsume': 'pub_Natsume',
 'pub_Nintendo': 'pub_Nintendo',
 'pub_Nippon_Ichi_Software': 'pub_Nippon_Ichi_Software',
 'pub_PQube': 'pub_PQube',
 'pub_Sega': 'pub_Sega',
 'pub_Sony_Computer_Entertainment': 'pub_Sony_Computer_Entertainment',
 'pub_Sony_Computer_Entertainment_America': 'pub_Sony_Computer_Entertainment_America',
 'pub_Sony_Computer_Entertainment_Europe': 'pub_Sony_Computer_Entertainment_Europe',
 'pub_Sony_Interactive_Entertainment': 'pub_Sony_Interactive_Entertainment',
 'pub_Square_Enix': 'pub_Square_Enix',
 'pub_Studio_Wildcard': 'pub_Studio_Wildcard',
 'pub_Take-Two_Interactive': 'pub_TakeTwo_Interactive',
 'pub_Tecmo_Koei': 'pub_Tecmo_Koei',
 'pub_Ubisoft': 'pub_Ubisoft',
 'pub_Warner_Bros._Interactive_Entertainment': 'pub_Warner_Bros_Interactive_Entertainment',
 'pub_Xseed_Games': 'pub_Xseed_Games',
 'gen_Action': 'gen_Action',
 'gen_Action-Adventure': 'gen_Action-Adventure',
 'gen_Adventure': 'gen_Adventure',
 'gen_Fighting': 'gen_Fighting',
 'gen_Misc': 'gen_Misc',
 'gen_Music': 'gen_Music',
 'gen_Party': 'gen_Party',
 'gen_Platform': 'gen_Platform',
 'gen_Puzzle': 'gen_Puzzle',
 'gen_Racing': 'gen_Racing',
 'gen_RolePlaying': 'gen_RolePlaying',
 'gen_Sandbox': 'gen_Sandbox',
 'gen_Shooter': 'gen_Shooter',
 'gen_Simulation': 'gen_Simulation',
 'gen_Sports': 'gen_Sports',
 'gen_Strategy': 'gen_Strategy',
 'con_3DS': 'con_3DS',
 'con_NS': 'con_NS',
 'con_PC': 'con_PC',
 'con_PS3': 'con_PS3',
 'con_PS4': 'con_PS4',
 'con_PSV': 'con_PSV',
 'con_WiiU': 'con_WiiU',
 'con_XOne': 'con_XOne'})

In [51]:
dummy = ''
for names in cleaned_dummies:
    dummy += names
    dummy += '+'
dummy

'pub_Activision+pub_Atlus+pub_Bandai_Namco_Entertainment+pub_Bethesda_Softworks+pub_Capcom+pub_Codemasters+pub_Deep_Silver+pub_Electronic_Arts+pub_Gearbox_Software+pub_Koch_Media+pub_Konami_Digital_Entertainment+pub_Level_5+pub_Marvelous_Interactive+pub_Maximum_Games+pub_Microsoft_Game_Studios+pub_Microsoft_Studios+pub_Mojang+pub_Namco_Bandai_Games+pub_Natsume+pub_Nintendo+pub_Nippon_Ichi_Software+pub_PQube+pub_Sega+pub_Sony_Computer_Entertainment+pub_Sony_Computer_Entertainment_America+pub_Sony_Computer_Entertainment_Europe+pub_Sony_Interactive_Entertainment+pub_Square_Enix+pub_Studio_Wildcard+pub_Take-Two_Interactive+pub_Tecmo_Koei+pub_Ubisoft+pub_Warner_Bros._Interactive_Entertainment+pub_Xseed_Games+gen_Action+gen_Action-Adventure+gen_Adventure+gen_Fighting+gen_Misc+gen_Music+gen_Party+gen_Platform+gen_Puzzle+gen_Racing+gen_RolePlaying+gen_RolePlaying+gen_Sandbox+gen_Shooter+gen_Simulation+gen_Sports+gen_Strategy+con_3DS+con_NS+con_PC+con_PS3+con_PS4+con_PSV+con_WiiU+con_XOne+'

In [67]:
form = ('All_Time_Sales~Weekly_Sales+Weeks_Out+pub_Activision+' +
        'pub_Capcom+pub_Electronic_Arts+pub_Gearbox_Software+' +
        'pub_Level_5+pub_Maximum_Games+pub_Microsoft_Game_Studios+' +
        'pub_Natsume+pub_Nintendo+' +
        'pub_PQube+pub_Sega+' +
        'pub_Sony_Computer_Entertainment+pub_Sony_Computer_Entertainment+pub_Square_Enix+pub_Studio_Wildcard+' +
        'pub_TakeTwo_Interactive+pub_Tecmo_Koei+pub_Ubisoft+pub_Warner_Bros_Interactive_Entertainment+pub_Xseed_Games+' +
        'gen_Action+gen_Action-Adventure+gen_Adventure+gen_Fighting+gen_Misc+gen_Music+gen_Party+' +
        'gen_Platform+gen_Racing+gen_RolePlaying+gen_RolePlaying+gen_Sandbox+gen_Shooter+' +
        'gen_Simulation+gen_Sports+gen_Strategy+con_3DS+con_NS+con_PC+con_PS3+con_PS4+con_WiiU+con_XOne')





In [68]:
lr_model = ols(formula=form, data=df2).fit()

In [69]:
lr_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         All_Time_Sales   R-squared:                       0.643
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     7.752
Date:                Thu, 18 Apr 2019   Prob (F-statistic):           9.15e-24
Time:                        15:09:42   Log-Likelihood:                -3639.7
No. Observations:                 229   AIC:                             7367.
Df Residuals:                     185   BIC:                             7519.
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                 -4.839e+05   1.92e+06     -0.252      0.801   -4.27e+06     3.3e+06
Weekly_Sales                                  9.5556      1.241      7.701      0.000       7.108      12.004
Weeks_Out                                  4.351e+04   3569.800     12.188      0.000    3.65e+04    5.06e+04
pub_Activision                             1.637e+06   8.03e+05      2.040      0.043    5.37e+04    3.22e+06
pub_Capcom                                 -1.21e+06   9.42e+05     -1.285      0.201   -3.07e+06    6.49e+05
pub_Electronic_Arts                         1.55e+06   6.59e+05      2.351      0.020    2.49e+05    2.85e+06
pub_Gearbox_Software                      -3.746e+06   1.61e+06     -2.322      0.021   -6.93e+06   -5.64e+05
pub_Level_5                               -9.734e+05   1.43e+06     -0.680      0.497    -3.8e+06    1.85e+06
pub_Maximum_Games                         -1.386e+06   2.26e+06     -0.614      0.540   -5.84e+06    3.06e+06
pub_Microsoft_Game_Studios                 1.283e+06   1.34e+06      0.955      0.341   -1.37e+06    3.93e+06
pub_Natsume                                1.651e+06   2.58e+06      0.640      0.523   -3.44e+06    6.74e+06
pub_Nintendo                               1.862e+06   6.92e+05      2.689      0.008    4.96e+05    3.23e+06
pub_PQube                                 -1.819e+06   2.21e+06     -0.822      0.412   -6.19e+06    2.55e+06
pub_Sega                                   -2.28e+06   8.95e+05     -2.546      0.012   -4.05e+06   -5.14e+05
pub_Sony_Computer_Entertainment            7.388e+05   8.01e+05      0.922      0.358   -8.42e+05    2.32e+06
pub_Square_Enix                           -6.194e+05   6.68e+05     -0.928      0.355   -1.94e+06    6.98e+05
pub_Studio_Wildcard                       -1.304e+06   2.21e+06     -0.589      0.556   -5.67e+06    3.06e+06
pub_TakeTwo_Interactive                    1.015e+06   7.78e+05      1.304      0.194   -5.21e+05    2.55e+06
pub_Tecmo_Koei                            -1.737e+06   1.61e+06     -1.082      0.281    -4.9e+06    1.43e+06
pub_Ubisoft                                9.525e+04    6.4e+05      0.149      0.882   -1.17e+06    1.36e+06
pub_Warner_Bros_Interactive_Entertainment -1.514e+05   8.23e+05     -0.184      0.854   -1.77e+06    1.47e+06
pub_Xseed_Games                            -7.79e+05   1.84e+06     -0.424      0.672    -4.4e+06    2.84e+06
gen_Action                                 1.575e+05   7.12e+05      0.221      0.825   -1.25e+06    1.56e+06
gen_Adventure                             -1.607e+06   9.81e+05     -1.639      0.103   -3.54e+06    3.28e+05
gen_Fighting                              -1.251e+06   8.77e+05     -1.427      0.155   -2.98e+06    4.79e+05
gen

In [64]:
df2 = df2.drop(['pub_Bandai_Namco_Entertainment','pub_Bethesda_Softworks','pub_Codemasters',
               'pub_Deep_Silver','pub_Koch_Media','pub_Konami_Digital_Entertainment', 'pub_Marvelous_Interactive',
               'pub_Mojang','pub_Microsoft_Studios','pub_Namco_Bandai_Games','pub_Nippon_Ichi_Software'
               ], axis=1)